# Advanced Scheduling

- We Could Add Priorities to favor some processes over others
    - Assign each process a **priority**
- Run higher priority processes first, round-robin processes of equal priority  
    - Can be preemptive or non-preemptive

### Priorities

##### ISSUE 1: lead lower priority processes to "starvation"

--> One solution is to have the OS dynamically change the priority. Older processes that haven’t been executed in a long time increase priority

##### ISSUE 2: **accidentally change** the priority of a low priority process to a high one 

This is caused by dependencies, e.g. a high priority depends on "output" from a low priority --> wrong order

One solution is <mark>**priority inheritance**</mark>:  
- Process providing data should be of higher priority
- Chain together multiple inheritances if needed  
- Revert back to the original priority after dependency

### **Foreground processes** and **Background processes** 

- <mark>Foreground processes</mark> are interactable and need good response time --> more priority
- <mark>Background processes</mark> may not need good response time, just throughput --> less priority

### Multiple Queues

We could create different queues for foreground and background processes: (Example:)
- Foreground uses RR
- Background uses FCFS

--> We have to schedule between queues
- RR between the queues
- Use a priority for each queue

..... The scheduling can get more and more complicated (There’s no “right answer”, only trade-offs)  

***

## Multiprocessor scheduling

Assume <mark>**symmetric multiprocessing (SMP)**</mark>:
- All CPUs are connected to the same physical memory
- The CPUs have their own private cache

### 1.  Use the Same Scheduling for All CPUs

There’s still only one scheduler --> **Keep scheduling processes across available CPUs**

Advantage:
- Good CPU utilization  
- Fair to all processes

Disadvantages:
- Not scalable (**everything blocks on global scheduler**)
- Poor cache locality (If a process gets RR and move to a different CPU, different cache access across different CPU --> no locality)

This was the approach in Linux 2.4

### 2. Create Per-CPU Schedulers

When there’s a new process, assign it to a CPU  
One strategy is to assign it to the CPU with the lowest number of processes  
**--> Process will not change CPU**

Advantages:
- Easy to implement
- Scalable (there’s no blocking on scheduler) (schedulers will not have to communicate with each other)
- Good cache locality (still on the same CPU)

Disadvantages:
- Load imbalance: Some CPUs may have less processe/less intensive ones

### 2+. Compromise between Global Scheduler and Per-CPU Scheduler

Keep a global scheduler that can rebalance per-CPU queues  
If a CPU is idle, take a process from another CPU and put to idle CPU (**work stealing** - "process stealing")

Disadvantage: If a process changes core, the cache locality will be reset

--> If we really want a process to have better preformance:

Use <mark>**processor affinity**</mark>  
The preference of a process to be scheduled on the same core

This is a simplified version of the O(1) scheduler in Linux 2.6

### 3. “Gang” Scheduling

Multiple processes may need to be scheduled simultaneously

The scheduler on each CPU cannot be completely independent

“Gang Scheduling” (**Coscheduling**)  
Allows you to run a set of processes simultaneously (acting as a unit)  

This requires a global context-switch across all CPUs (all processes at the same time)  
(will not have to deal with this in this course)

### 4. Real-Time Scheduling

Real-time means there are time constraints (e.g. audio, autopilot --> Needs to be done fast)

- A hard real-time system: Required to guarantee a task completes within a certain amount of time
- A soft real-time system: Critical processes have a higher priority and the deadline is met in practice

Linux is an example of soft real-time

### 5. Linux scheduling - O(1) scheduler

#### 5.1. Real-time processes - Same scheduler for all CPUs (in Kernel - FCFS or RR)

Soft Real-Time Processes Are Always Prioritized over Normal Processes

**Between soft real-time processes: Always schedule the highest priority processes first**

Use a multilevel queue scheduler for processes with the same priority  
Also let the OS dynamically adjust the priority

The soft real-time scheduling policy will either be SCHED_FIFO or SCHED_RR (because they are predictable)  
There are 100 static priority levels

#### 5.2. Normal processes - Per-CPU Schedulers

**Between normal processes: Adjust the priority based on aging (later)**

Normal scheduling policies apply to the other processes (SCHED_NORMAL)  
There are 40 Normal priority levels

**Linux tries to unifies all the normal and real-time process**

--> This is the number that's shown through Linux commands

Processes can change their own priorities with system calls:  
`nice`, `sched_setscheduler`

![Alt text](images/image13.png)

If you read the `PRI` of all processes in Linux, showing `RT` means -100

### Linux Scheduler Evolution

2.4—2.6, a O(N) global queue  
Simple, but poor performance with multiprocessors and many processes

(5. is here)  
2.6—2.6.22, a per-CPU run queue, O(1) scheduler  
Complex to get right, interactivity had issues  
No guarantee of fairness

2.6.23—Present, the **completely fair scheduler (CFS)**  
Fair, and allows for good interactivity

### The O(1) Scheduler Has Issues with Modern Processes

- Foreground and background processes are a good division  
    - Easier with a terminal, but less obvious with GUI processes

- Now the kernel has to detect interactive processes with heuristics (**a guess** --> might be unfair)  
    - Processes that sleep a lot may be more interactive  
    - This is ad hoc, and could be unfair

##### --> How would we introduce fairness for different priority processes?  

- Use different size time slices based on priority
    - The higher the priority, the larger the time slice --> could scale time slice depending on how "fair" we have been on that process in the past
    - There are also situations where this ad hoc solution could be unfair

***
## 6. Ideal Fair Scheduling

Assume you have an infinitely small time slice  
If you have n processes, each runs at 1/n rate  

![Alt text](images/image14.png)

CPU usage is divided equally among every process

### Example IFS Scheduling

Consider the following processes: In every time units, all processes have perfectly fair share

![Alt text](images/image15.png) ![Alt text](images/image16.png)

#### THIS IS IMPRACTICAL

- Too many context switches, running scheduling algorithm too many times
- You have to constantly scan all processes (to check if every processes have a fair share), which is O(N) --> Super slow

## 7. Completely Fair Scheduler (CFS) - NOW

- For each runnable process, assign it a “virtual runtime”
    - At each scheduling point where the process runs for time t
        - Increase the virtual runtime by <mark>**t (actual running time) × weight (based on priority)**</mark>
        - If it is a **higher priority task, weight is lower** --> less time to run
- The virtual runtime monotonically increases: only increases (never decreases)
    - Scheduler selects the process based on the lowest virtual runtime --> will run this process
    - Compute its dynamic time slice based on the IFS

Allow the process to run, when the time slice ends repeat the process

### CFS is Implemented with Red-Black Trees

- Keyed by virtual runtime
- O(1) to find minimum --> the one that will run next

The implementation uses a red-black tree with nanosecond granularity  
Doesn’t need to guess the interactivity of a process

CFS tends to favour I/O bound processes by default (they will usually go to sleep, so they will have small virtual runtime)  
Small CPU bursts translate to a low virtual runtime  
It will get a larger time slice, in order to catch up to the ideal  

### Scheduling Gets Even More Complex

There are more solutions, and more issues:
- Introducing priority also introduces priority inversion
- Some processes need good interactivity, others not so much
- Multiprocessors may require per-CPU queues
- Real-time requires predictability
- Completely Fair Scheduler (CFS) tries to model the ideal fairness